In [1]:
%%capture
!pip install datasets
!pip install transformers
!pip install torch

In [13]:
from transformers import pipeline
from torch.utils.data import Dataset
from tqdm.auto import tqdm
from datasets import load_dataset

########################################################################################################################
# This script needs GPU acceleration e.g. via Google Colab
########################################################################################################################

# Create a pipeline using the desired model and tokenizer using a GPU
pipe = pipeline(model="ProsusAI/finbert", tokenizer='ProsusAI/finbert', device=0)


# Define dataset structure
class NewsDataset(Dataset):
    def __init__(self, original_list):
        self.original_list = original_list

    def __len__(self):
        return len(self.original_list)

    def __getitem__(self, i):
        return self.original_list[i]


# Load dataset
d = load_dataset("csv", data_files="/content/drive/MyDrive/masterProject/av_clean.csv")
d = d["train"]

# Instantiate dataset
dataset = NewsDataset(d["summary"])

# # Find optimal batch size on subset of data
# for batch_size in [1, 8, 64, 256]:
#     print("-" * 30)
#     print(f"Streaming batch_size={batch_size}")
#     for out in tqdm(pipe(dataset, batch_size=batch_size), total=len(dataset)):
#         pass


# Label text data
results = []
for out in tqdm(pipe(dataset, batch_size=8), total=len(dataset)):
    results.append(out["label"])

# Add results to the original dataset
d = d.add_column("finBERT", results)

# Save the DataFrame as a CSV file
d.to_csv("/content/drive/MyDrive/masterProject/av_labelled.csv", columns=["date", "summary", "stock", "finBERT"], index=False)

  0%|          | 0/5000 [00:00<?, ?it/s]

Creating CSV from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

1031459